# Training Notebook

## Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import cv2
import albumentations as A
import torchvision.models as models
import os
from tqdm.notebook import tqdm
import time

## Resize and save the image

In [2]:
def resize_img(input_path, write_path, size=(256, 256)):
    img = cv2.imread(input_path)
    img = cv2.resize(img, size)
    cv2.imwrite(write_path, img)

## Augmentations pipeline

In [3]:
# Define transformations
transforms = A.Compose([
    A.Flip(p=0.5),
    A.Rotate(limit=10, 
             border_mode=cv2.BORDER_CONSTANT, 
             value=0.0, p=0.75),
    A.RandomResizedCrop(width=224, height=224, scale=(0.33, 1), p=1),
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225), 
                max_pixel_value=255.0, 
                p=1.0)
])

## Dataset

In [4]:
class ArtDataset(Dataset):
    def __init__(self, df, label_dict, transforms):
        self.df = df
        self.transforms = transforms
        self.label_dict = label_dict
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # Get filename and label
        filename = row['filename']
        label = torch.LongTensor(self.label_dict[row['label']])
        # Read image, correct color channels
        img = cv2.imread(filename)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Augmentations
        transformed = self.transforms(image=img.astype(np.uint8))
        img = transformed['image']
        # Convert to array, switch channels, scale, return
        # img = np.transpose(img, (2, 0, 1))
        img = torch.tensor(img).float()
        return img, label

## Transfer learning functions

In [5]:
def set_classification_layer(model, model_type='vgg', num_classes=25):
    if model_type == 'vgg':
        model.classifier = nn.Sequential(
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=num_classes, bias=True)
        )
    elif model_type == 'resnet':
        model.fc = nn.Linear(in_features=4096, out_features=num_classes, bias=True)
    elif model_type == 'vit':
        model.heads = nn.Linear(in_features=768, out_features=num_classes, bias=True)
    elif model_type == 'convnext':
        model.classifier = nn.Sequential(
            nn.LayerNorm2d((768,), eps=1e-06, elementwise_affine=True),
            nn.Flatten(start_dim=1, end_dim=-1),
            nn.Linear(in_features=768, out_features=num_classes, bias=True)
        )
    else:
        print(f'Unknown model_type {model_type}. Acceptable types are: "vgg", "resnet", "vit", or "convnext"')
    

In [6]:
def freeze_model(model, **classargs):
    '''
    Given an existing model, freeze pre-trained weights and
    re-instantiate the classifier.
    '''
    # Freeze all parameters
    for param in model.parameters():
        param.requires_grad = False
    # Re-instantiate the classifier head
    set_classification_layer(model, **classargs)

In [7]:
vgg = models.vgg16()
for name, param in vgg.named_parameters():
    print(f"{name} gradient is set to", param.requires_grad)

features.0.weight gradient is set to True
features.0.bias gradient is set to True
features.2.weight gradient is set to True
features.2.bias gradient is set to True
features.5.weight gradient is set to True
features.5.bias gradient is set to True
features.7.weight gradient is set to True
features.7.bias gradient is set to True
features.10.weight gradient is set to True
features.10.bias gradient is set to True
features.12.weight gradient is set to True
features.12.bias gradient is set to True
features.14.weight gradient is set to True
features.14.bias gradient is set to True
features.17.weight gradient is set to True
features.17.bias gradient is set to True
features.19.weight gradient is set to True
features.19.bias gradient is set to True
features.21.weight gradient is set to True
features.21.bias gradient is set to True
features.24.weight gradient is set to True
features.24.bias gradient is set to True
features.26.weight gradient is set to True
features.26.bias gradient is set to True


In [8]:
# Freeze model
freeze_model(vgg, num_classes=25, model_type='vgg')
vgg.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=25, bias=True)
)

In [9]:
for name, param in vgg.named_parameters():
    print(f"{name} gradient is set to", param.requires_grad)

features.0.weight gradient is set to False
features.0.bias gradient is set to False
features.2.weight gradient is set to False
features.2.bias gradient is set to False
features.5.weight gradient is set to False
features.5.bias gradient is set to False
features.7.weight gradient is set to False
features.7.bias gradient is set to False
features.10.weight gradient is set to False
features.10.bias gradient is set to False
features.12.weight gradient is set to False
features.12.bias gradient is set to False
features.14.weight gradient is set to False
features.14.bias gradient is set to False
features.17.weight gradient is set to False
features.17.bias gradient is set to False
features.19.weight gradient is set to False
features.19.bias gradient is set to False
features.21.weight gradient is set to False
features.21.bias gradient is set to False
features.24.weight gradient is set to False
features.24.bias gradient is set to False
features.26.weight gradient is set to False
features.26.bias g

## Training functions

In [11]:
# Dictionary for easily passing training arguments
training_params = {'epochs': 10,
                  'batch_size': 16,
                  'loss_fct': nn.CrossEntropyLoss()}

def eval_model(model, dl, training_params):
    # Get GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # Evaluate
    model.eval()
    total_loss = 0
    total_obs = 0
    total_correct = 0
    loss_fct = training_params['loss_fct']
    for X, y in dl:
        n_obs = len(y)
        # Forward pass and calculate loss
        yhat = model(X.to(device))#.softmax(dim=1)
        loss = loss_fct(yhat.to(device), y.to(device))
        total_loss += n_obs * loss.item()
        total_obs += n_obs
        # Calculate batch accuracy
        ypred = np.argmax(yhat.cpu().detach().numpy(), axis=1)
        y_arr = y.detach().numpy()
        total_correct += n_obs * accuracy_score(y_arr, ypred)
    # Return loss, accuracy
    avg_loss = total_loss / total_obs
    accuracy = total_correct / total_obs
    return avg_loss, accuracy
    
    
def train_model(model, optimizer, scheduler, train_ds, valid_ds, training_params):
    # Get loss function
    loss_fct = training_params['loss_fct']
    # Create dataloaders based on batch size
    batch_size = trainin_params['batch_size']
    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    valid_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=False)
    # Get GPU if available
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    # Train
    for _ in range(training_params['epochs']):
        # Put model in train mode
        model.train()
        # Train on training dataloader
        for X, y in train_dl:
            # Clear gradients
            optimizer.zero_grad()
            # Forward pass and loss calculation
            yhat = model(X.to(device))#.softmax(dim=1)
            loss = loss_fct(yhat.to(device), y.to(device))
            # Backward pass and step
            loss.backward()
            optimizer.step()
            scheduler.step()
        # Calculate loss, accuracy on train and validation
        train_loss, train_acc = eval_model(model, train_dl)
        valid_loss, valid_acc = eval_model(model, valid_dl)
        train_str = f"train loss: {train_loss:.4f} | train acc: {train_acc:.4f}"
        valid_str = f" | valid loss: {valid_loss:.4f} | valid acc: {valid_acc:.4f}"
        print(f'[{_}] ' + train_str + valid_str)
    

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [26]:
artworks = pd.read_csv('artworks.csv')
artworks.head(3).T

,0,1,2
Unnamed: 0,0,1,2
id,57728479edc2cb3880fdea33,57728a62edc2cb388010efa1,57728a61edc2cb388010ef5f
title,Silhouette fantastique,First Communion of Anaemic Young Girls in the ...,Apoplectic Cardinals Harvesting Tomatoes on th...
year,1854,1883,1884
width,500,1324,1400
height,366,848,980
artistName,Victor Hugo,Alphonse Allais,Alphonse Allais
image,https://uploads0.wikiart.org/images/victor-hug...,https://uploads0.wikiart.org/images/alphonse-a...,https://uploads2.wikiart.org/images/alphonse-a...
map,0*23**67*,01234*67*,01234*67*
paintingUrl,/en/victor-hugo/silhouette-fantastique-1854,/en/alphonse-allais/first-communion-of-anaemic...,/en/alphonse-allais/apoplectic-cardinals-harve...


In [27]:
artworks[artworks['style']=='abstract-art'].head()

,Unnamed: 0,id,title,year,width,height,artistName,image,map,paintingUrl,artistUrl,albums,flags,images,style
0,0,57728479edc2cb3880fdea33,Silhouette fantastique,1854,500,366,Victor Hugo,https://uploads0.wikiart.org/images/victor-hug...,0*23**67*,/en/victor-hugo/silhouette-fantastique-1854,/en/victor-hugo,NaN,2,NaN,abstract-art
1,1,57728a62edc2cb388010efa1,First Communion of Anaemic Young Girls in the ...,1883,1324,848,Alphonse Allais,https://uploads0.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/first-communion-of-anaemic...,/en/alphonse-allais,NaN,2,NaN,abstract-art
2,2,57728a61edc2cb388010ef5f,Apoplectic Cardinals Harvesting Tomatoes on th...,1884,1400,980,Alphonse Allais,https://uploads2.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/apoplectic-cardinals-harve...,/en/alphonse-allais,NaN,2,NaN,abstract-art
3,3,57728a61edc2cb388010ef71,Band of Greyfriars in the Fog (Band Of Dusty D...,1884,1400,980,Alphonse Allais,https://uploads2.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/band-of-greyfriars-in-the-...,/en/alphonse-allais,NaN,2,NaN,abstract-art
4,4,57728a61edc2cb388010ef81,Negroes Fighting in a Tunnel by Night,1884,800,560,Alphonse Allais,https://uploads5.wikiart.org/images/alphonse-a...,0123**67*,/en/alphonse-allais/negroes-fighting-in-a-tunn...,/en/alphonse-allais,NaN,2,NaN,abstract-art


In [28]:
artworks['painting_str'] = artworks['paintingUrl'].apply(lambda x: f"{x.split('/')[-1]}.jpg")
artworks.head()

,Unnamed: 0,id,title,year,width,height,artistName,image,map,paintingUrl,artistUrl,albums,flags,images,style,painting_str
0,0,57728479edc2cb3880fdea33,Silhouette fantastique,1854,500,366,Victor Hugo,https://uploads0.wikiart.org/images/victor-hug...,0*23**67*,/en/victor-hugo/silhouette-fantastique-1854,/en/victor-hugo,NaN,2,NaN,abstract-art,silhouette-fantastique-1854.jpg
1,1,57728a62edc2cb388010efa1,First Communion of Anaemic Young Girls in the ...,1883,1324,848,Alphonse Allais,https://uploads0.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/first-communion-of-anaemic...,/en/alphonse-allais,NaN,2,NaN,abstract-art,first-communion-of-anaemic-young-girls-in-the-...
2,2,57728a61edc2cb388010ef5f,Apoplectic Cardinals Harvesting Tomatoes on th...,1884,1400,980,Alphonse Allais,https://uploads2.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/apoplectic-cardinals-harve...,/en/alphonse-allais,NaN,2,NaN,abstract-art,apoplectic-cardinals-harvesting-tomatoes-on-th...
3,3,57728a61edc2cb388010ef71,Band of Greyfriars in the Fog (Band Of Dusty D...,1884,1400,980,Alphonse Allais,https://uploads2.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/band-of-greyfriars-in-the-...,/en/alphonse-allais,NaN,2,NaN,abstract-art,band-of-greyfriars-in-the-fog-band-of-dusty-dr...
4,4,57728a61edc2cb388010ef81,Negroes Fighting in a Tunnel by Night,1884,800,560,Alphonse Allais,https://uploads5.wikiart.org/images/alphonse-a...,0123**67*,/en/alphonse-allais/negroes-fighting-in-a-tunn...,/en/alphonse-allais,NaN,2,NaN,abstract-art,negroes-fighting-in-a-tunnel-by-night.jpg


In [29]:
abstract = artworks[artworks['style']=='abstract-art']
abstract.sort_values('painting_str')

,Unnamed: 0,id,title,year,width,height,artistName,image,map,paintingUrl,artistUrl,albums,flags,images,style,painting_str
1373,1373,577283dfedc2cb3880fbfa84,0.42 - 69,1969,640,433,Fernando Lanhas,https://uploads7.wikiart.org/images/fernando-l...,0*23**67*,/en/fernando-lanhas/0-42-69-1969,/en/fernando-lanhas,NaN,2,NaN,abstract-art,0-42-69-1969.jpg
1905,1905,59f38860edc2c9ac90c7c44c,03.06.13,2013,662,657,Andrzej Nowacki,https://uploads4.wikiart.org/00155/images/andr...,0123**67*,/en/andrzej-nowacki/03-06-13,/en/andrzej-nowacki,NaN,2,NaN,abstract-art,03-06-13.jpg
1965,1965,59f38d6eedc2c9ac90df5975,03.11.16-03,2016,881,1066,Andrzej Nowacki,https://uploads4.wikiart.org/00155/images/andr...,01234*67*,/en/andrzej-nowacki/03-11-16-03,/en/andrzej-nowacki,NaN,2,NaN,abstract-art,03-11-16-03.jpg
1964,1964,59f38d03edc2c9ac90dd8f18,03.11.16-11,2016,942,1156,Andrzej Nowacki,https://uploads4.wikiart.org/00155/images/andr...,01234*67*,/en/andrzej-nowacki/03-11-16-11,/en/andrzej-nowacki,NaN,2,NaN,abstract-art,03-11-16-11.jpg
1962,1962,59f38c2fedc2c9ac90d9eb96,03.11.16-20,2016,2835,2835,Andrzej Nowacki,https://uploads7.wikiart.org/00155/images/andr...,01234567*,/en/andrzej-nowacki/03-11-16-20,/en/andrzej-nowacki,NaN,2,NaN,abstract-art,03-11-16-20.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,1627,5c18df93edc2c914807d33c1,Untitled Composition #1,1989,600,431,Yuri Zlotnikov,https://uploads2.wikiart.org/00215/images/yuri...,0*23**67*,/en/yuriy-zlotnikov/zlotnikov-7,/en/yuriy-zlotnikov,NaN,2,NaN,abstract-art,zlotnikov-7.jpg
1226,1226,5c18df93edc2c914807d33c3,Signal series,1962,304,430,Yuri Zlotnikov,https://uploads6.wikiart.org/00215/images/yuri...,**23**67*,/en/yuriy-zlotnikov/zlotnikov-8,/en/yuriy-zlotnikov,NaN,2,NaN,abstract-art,zlotnikov-8.jpg
1069,1069,5c18df93edc2c914807d33ad,Signal series,1956,500,354,Yuri Zlotnikov,https://uploads6.wikiart.org/00215/images/yuri...,0*23**67*,/en/yuriy-zlotnikov/zlotnikov,/en/yuriy-zlotnikov,NaN,2,NaN,abstract-art,zlotnikov.jpg
1910,1910,59510010edc2c9bc40f1d7f6,Zwei,2014,651,850,Daniel Sambo-Richter,https://uploads7.wikiart.org/00173/images/dani...,01234*67*,/en/daniel-sambo-richter/zwei-2014,/en/daniel-sambo-richter,NaN,2,NaN,abstract-art,zwei-2014.jpg


In [30]:
artworks['filepath'] = artworks.apply(lambda row: row['style']+"/"+row['painting_str'], axis=1)
artworks.head()

,Unnamed: 0,id,title,year,width,height,artistName,image,map,paintingUrl,artistUrl,albums,flags,images,style,painting_str,filepath
0,0,57728479edc2cb3880fdea33,Silhouette fantastique,1854,500,366,Victor Hugo,https://uploads0.wikiart.org/images/victor-hug...,0*23**67*,/en/victor-hugo/silhouette-fantastique-1854,/en/victor-hugo,NaN,2,NaN,abstract-art,silhouette-fantastique-1854.jpg,abstract-art/silhouette-fantastique-1854.jpg
1,1,57728a62edc2cb388010efa1,First Communion of Anaemic Young Girls in the ...,1883,1324,848,Alphonse Allais,https://uploads0.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/first-communion-of-anaemic...,/en/alphonse-allais,NaN,2,NaN,abstract-art,first-communion-of-anaemic-young-girls-in-the-...,abstract-art/first-communion-of-anaemic-young-...
2,2,57728a61edc2cb388010ef5f,Apoplectic Cardinals Harvesting Tomatoes on th...,1884,1400,980,Alphonse Allais,https://uploads2.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/apoplectic-cardinals-harve...,/en/alphonse-allais,NaN,2,NaN,abstract-art,apoplectic-cardinals-harvesting-tomatoes-on-th...,abstract-art/apoplectic-cardinals-harvesting-t...
3,3,57728a61edc2cb388010ef71,Band of Greyfriars in the Fog (Band Of Dusty D...,1884,1400,980,Alphonse Allais,https://uploads2.wikiart.org/images/alphonse-a...,01234*67*,/en/alphonse-allais/band-of-greyfriars-in-the-...,/en/alphonse-allais,NaN,2,NaN,abstract-art,band-of-greyfriars-in-the-fog-band-of-dusty-dr...,abstract-art/band-of-greyfriars-in-the-fog-ban...
4,4,57728a61edc2cb388010ef81,Negroes Fighting in a Tunnel by Night,1884,800,560,Alphonse Allais,https://uploads5.wikiart.org/images/alphonse-a...,0123**67*,/en/alphonse-allais/negroes-fighting-in-a-tunn...,/en/alphonse-allais,NaN,2,NaN,abstract-art,negroes-fighting-in-a-tunnel-by-night.jpg,abstract-art/negroes-fighting-in-a-tunnel-by-n...
